In [23]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import re

import tensorflow as tf
from tensorflow.keras.layers import Layer, Lambda, Conv2D, Dense, MaxPooling2D, Input, Flatten, Dropout, BatchNormalization, LeakyReLU, Activation, Add
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

In [2]:
!unzip '/content/drive/MyDrive/bounding_box_train.zip' -d 'trainset'

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: trainset/bounding_box_train/0788_c6s2_120318_05.jpg  
  inflating: trainset/bounding_box_train/0788_c6s2_120368_01.jpg  
  inflating: trainset/bounding_box_train/0788_c6s2_120418_01.jpg  
  inflating: trainset/bounding_box_train/0788_c6s2_120443_01.jpg  
  inflating: trainset/bounding_box_train/0792_c1s4_026706_01.jpg  
  inflating: trainset/bounding_box_train/0792_c1s4_026731_01.jpg  
  inflating: trainset/bounding_box_train/0792_c1s4_026881_02.jpg  
  inflating: trainset/bounding_box_train/0792_c2s2_084057_03.jpg  
  inflating: trainset/bounding_box_train/0792_c2s2_084157_03.jpg  
  inflating: trainset/bounding_box_train/0792_c3s2_087428_01.jpg  
  inflating: trainset/bounding_box_train/0792_c3s2_087503_02.jpg  
  inflating: trainset/bounding_box_train/0792_c3s2_087528_01.jpg  
  inflating: trainset/bounding_box_train/0792_c4s4_027135_01.jpg  
  inflating: trainset/bounding_box_train/0792_c4s4_027160_02.j

# Siamese Network

I use a Siamese network to identify people. This enables the subsequent recognition and unique identification of people in successive frames. The Siamese network is specifically designed to capture patterns and features in images, helping to highlight the similarities or differences between different people. Using this technique, people can be tracked and identified across different frames, even if their position or pose changes. This is particularly useful for ensuring consistent tracking of people in video sequences and ensuring a clear association between different frames.

In [3]:
def extract_first_number(filename):
    # this function extracts the first sequence of digits from the image name. I use it to identify the same people later.
    match = re.search(r'\d+', filename)

    if match:
        return int(match.group())
    else:
        return None

def resize_and_pad_image(img, target_size=(64, 64)):
    # this function resizes the image to the target size and pads it with zeros to keep the aspect ratio
    height, width, _ = img.shape

    target_height, target_width = target_size
    aspect_ratio = width / height

    if width == height:
        return cv2.resize(img, target_size)

    if aspect_ratio > 1:
        new_width = target_width
        new_height = int(target_width / aspect_ratio)
    else:
        new_width = int(target_height * aspect_ratio)
        new_height = target_height

    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)

    top_pad = (target_height - new_height) // 2
    bottom_pad = target_height - new_height - top_pad
    left_pad = (target_width - new_width) // 2
    right_pad = target_width - new_width - left_pad

    padded_img = cv2.copyMakeBorder(resized_img, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    return padded_img

## Build Train, Validation and Test Partition

In [4]:
def load_data(path = './trainset/bounding_box_train/'):
    # this function loads the Images from the folder and save there labels
    files = os.listdir(path)
    files.sort()
    data = []
    counter = 0
    count_per_person = {}
    label = []
    for file in files:
            person_id = file.split('_')[0]

            if person_id not in count_per_person:
                count_per_person[person_id] = 1
            elif count_per_person[person_id] < 30:
                count_per_person[person_id] += 1
            else:
                continue 

            img = cv2.imread(path + file)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img/255.0

            img_resized = resize_and_pad_image(img, target_size=(64, 64))

            data.append(img_resized)
            label.append(extract_first_number(file))

            counter += 1
            if counter%1000 == 0:
              print("Verarbeitung von 1000 Bildern abgeschlossen.")
            if counter == 12000:
                break
    return data, label

In [11]:
images, labels = load_data()

Verarbeitung von 1000 Bildern abgeschlossen.
Verarbeitung von 1000 Bildern abgeschlossen.
Verarbeitung von 1000 Bildern abgeschlossen.
Verarbeitung von 1000 Bildern abgeschlossen.
Verarbeitung von 1000 Bildern abgeschlossen.
Verarbeitung von 1000 Bildern abgeschlossen.
Verarbeitung von 1000 Bildern abgeschlossen.
Verarbeitung von 1000 Bildern abgeschlossen.
Verarbeitung von 1000 Bildern abgeschlossen.
Verarbeitung von 1000 Bildern abgeschlossen.
Verarbeitung von 1000 Bildern abgeschlossen.
Verarbeitung von 1000 Bildern abgeschlossen.


In [18]:
def create_random_pairs(images, labels, num_positive_pairs=1000, num_negative_pairs=1000):
    pairs = []
    labels = np.array(labels)
    labels_pair = []

    unique_labels = np.unique(labels)

    for _ in range(num_positive_pairs):
        person_label = np.random.choice(unique_labels)

        pair_indices = np.random.choice(np.where(labels == person_label)[0], 2, replace=False)

        pairs.append([images[pair_indices[0]], images[pair_indices[1]]])
        labels_pair = np.append(labels_pair, 1)

    for _ in range(num_negative_pairs):
        person1, person2 = np.random.choice(unique_labels, 2, replace=False)

        indices1 = np.random.choice(np.where(labels == person1)[0])
        indices2 = np.random.choice(np.where(labels == person2)[0])
        
        pairs.append([images[indices1], images[indices2]])
        labels_pair = np.append(labels_pair, 0)

    return np.array(pairs), labels_pair

In [19]:
pairs, pair_labels = create_random_pairs(images, labels, num_positive_pairs=5000, num_negative_pairs=5000)

In [ ]:
pairs_train, pairs_temp, labels_train, labels_temp = train_test_split(pairs, pair_labels, test_size=0.3, random_state=42)
pairs_val, pairs_test, labels_val, labels_test = train_test_split(pairs_temp, labels_temp, test_size=0.5, random_state=42)

In [ ]:
len(pairs_train)

7000

In [ ]:
len(pairs_val)

1500

In [ ]:
len(pairs_test)

1500

# Create the custom Siamese Network

In [ ]:
def create_siamese_network(input_shape):
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    model = Sequential()
    model.add(Conv2D(64, (3, 3), input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.1))

    model.add(Conv2D(128, (3,3)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.1))

    model.add(Conv2D(256, (3,3)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.1))

    model.add(Conv2D(256, (3, 3)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.1))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))

    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))

    encoded_a = model(input_a)
    encoded_b = model(input_b)

    l1_distance = Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([encoded_a, encoded_b])

    prediction = Dense(1, activation='sigmoid')(l1_distance)

    siamese_model = Model(inputs=[input_a, input_b], outputs=prediction)

    return siamese_model

In [ ]:
input_shape = (64, 64, 3)
siamese_model = create_siamese_network(input_shape)
siamese_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 sequential (Sequential)     (None, 512)                  3854976   ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 lambda (Lambda)             (None, 512)                  0         ['sequential[0][0]',      

In [ ]:
siamese_model.compile(loss='binary_crossentropy',
                      optimizer=Adam(learning_rate=0.001),
                      metrics=['accuracy'])

# Train the Siamese Network

In [ ]:
siamese_model.fit([pairs_train[:, 0], pairs_train[:, 1]], labels_train,
                  epochs=100, batch_size=8,
                  validation_data=([pairs_val[:, 0], pairs_val[:, 1]], labels_val))

Epoch 1/100
875/875 [==============================] - 15s 17ms/step - loss: 0.0547 - accuracy: 0.9800 - val_loss: 0.4316 - val_accuracy: 0.8767
Epoch 2/100
875/875 [==============================] - 15s 17ms/step - loss: 0.0621 - accuracy: 0.9781 - val_loss: 0.3753 - val_accuracy: 0.9020
Epoch 3/100
875/875 [==============================] - 14s 16ms/step - loss: 0.0510 - accuracy: 0.9829 - val_loss: 0.3837 - val_accuracy: 0.8927
Epoch 4/100
875/875 [==============================] - 14s 16ms/step - loss: 0.0435 - accuracy: 0.9850 - val_loss: 0.4087 - val_accuracy: 0.8893
Epoch 5/100
875/875 [==============================] - 14s 16ms/step - loss: 0.0576 - accuracy: 0.9796 - val_loss: 0.4070 - val_accuracy: 0.8847
Epoch 6/100
875/875 [==============================] - 14s 16ms/step - loss: 0.0478 - accuracy: 0.9821 - val_loss: 0.4171 - val_accuracy: 0.8840
Epoch 7/100
875/875 [==============================] - 14s 16ms/step - loss: 0.0422 - accuracy: 0.9854 - val_loss: 0.3943 - val_ac

# Save the Siamese Network

In [ ]:
siamese_model.save('siamese_model_2.h5')

In [ ]:
siamese_model.save_weights('siamese_model_weights_2.h5')

In [ ]:
siamese_model.save('siamese_model_2.keras')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
from keras.src.saving import serialization_lib
serialization_lib.enable_unsafe_deserialization()

In [ ]:
loaded_model = load_model('/content/drive/MyDrive/siamese_model_2.keras', custom_objects={'safe_mode': False})

# Test the Siamese Network

In [ ]:
validation_loss, validation_accuracy = loaded_model.evaluate([pairs_val[:, 0], pairs_val[:, 1]], labels_val)
print(f'Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}')

test_loss, test_accuracy = loaded_model.evaluate([pairs_test[:, 0], pairs_test[:, 1]], labels_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

47/47 [==============================] - 6s 25ms/step - loss: 0.4895 - accuracy: 0.8800
Validation Loss: 0.4894562065601349, Validation Accuracy: 0.8799999952316284
47/47 [==============================] - 1s 11ms/step - loss: 0.5313 - accuracy: 0.8780
Test Loss: 0.5312526226043701, Test Accuracy: 0.878000020980835
